In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
!ls -a

.   .cache   datalab   hhh.csv	 .keras    .local  .rnd      well_parsed.csv
..  .config  .forever  .ipython  last.csv  .nv	   test.csv


In [0]:
import pandas as pd
import numpy as np

In [0]:
list_phonems = ['_', 'AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', \
                'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', \
                'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH']

In [8]:
list_phonems[9]

'D'

In [0]:
incoder_phonems = {list_phonems[i] : i for i in range(len(list_phonems))}

In [0]:
alphabet = [chr(ord('A') + i - 1) for i in range(27)]
alphabet.append('\'')
alphabet.append('-')

In [0]:
incoder_alph = {alphabet[i] : i for i in range(len(alphabet))}

In [12]:
incoder_alph['A']

1

In [0]:
data = drive.CreateFile({'id': '1gB9Q-EStnVdbCp5MMsonqmI_0rbb1sgh'}).GetContentString().split('\n')

In [14]:
data[0:2]

['LEMIEUX L_AH_M_Y_UW', 'MINDING M_AY_N_D_IH_NG']

In [0]:
def lens(data):
  LEN, MAX_INPUT_LEN, MAX_OUTPUT_LEN = 0, 0, 0
  for line in data:
    info = line.split(' ')
    MAX_INPUT_LEN = max(len(info[0]), MAX_INPUT_LEN)
    for lb in info[1:]:
      MAX_OUTPUT_LEN = max(MAX_OUTPUT_LEN, len(lb.split('_')))
      LEN += 1
  return LEN, MAX_INPUT_LEN, MAX_OUTPUT_LEN

In [0]:
LEN, MAX_INPUT_LEN, MAX_OUTPUT_LEN = lens(data)

In [17]:
LEN

89056

In [0]:
train = np.zeros((LEN, MAX_INPUT_LEN, len(alphabet)))
labels = np.zeros((LEN, MAX_INPUT_LEN, len(list_phonems)))
i = 0
for line in data:
  info = line.split(' ')
  for lb in info[1:]:
    j = 0
    for letter in info[0]:
      train[i, j] = np.array([int(i == incoder_alph[letter]) for i in range(len(alphabet))])
      j += 1
    j = 0
    for phonem in lb.split('_'):
      labels[i, j] = np.array([int(i == incoder_phonems[phonem]) for i in range(len(list_phonems))])
      j += 1
    while (j != MAX_INPUT_LEN):
      labels[i, j, 0] = 1
      j += 1
    i += 1

In [19]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        1.9G        5.3G        247M        5.5G         10G
Swap:            0B          0B          0B


In [20]:
train[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [21]:
import os
import sys
import numpy as np
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Activation, RepeatVector, TimeDistributed, LSTM, Reshape
from keras.layers import Conv1D, MaxPooling1D, Embedding, LeakyReLU
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Masking, Dense
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import RMSprop
from keras.layers import Bidirectional, GRU

HIDDEN_SIZE = 256

model = Sequential()
'''
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAX_INPUT_LEN, len(alphabet))))
model.add(RepeatVector(MAX_OUTPUT_LEN))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
model.add(TimeDistributed(Dense(len(list_phonems))))
model.add(Activation('softmax'))
'''
model.add(Masking(mask_value=0., input_shape=(MAX_INPUT_LEN, len(alphabet))))
model.add(Bidirectional(LSTM(256, return_sequences=True, recurrent_dropout=0.01,
                             activation='tanh', recurrent_activation='hard_sigmoid'), merge_mode="sum"))
model.add(LSTM(256, return_sequences=True, activation='tanh', recurrent_activation='hard_sigmoid'))
model.add(GRU(256, return_sequences=True, recurrent_activation='hard_sigmoid'))
model.add(GRU(256, return_sequences=True, recurrent_activation='hard_sigmoid'))
model.add(TimeDistributed(Dense(256, activation='relu')))
#model.add(Dense(MAX_OUTPUT_LEN * len(list_phonems), activation = 'relu'))
#model.add(Reshape((MAX_OUTPUT_LEN, len(list_phonems))))
model.add(TimeDistributed(Dense(len(list_phonems), activation='softmax')))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 34, 29)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 34, 256)           585728    
_________________________________________________________________
lstm_2 (LSTM)                (None, 34, 256)           525312    
_________________________________________________________________
gru_1 (GRU)                  (None, 34, 256)           393984    
_________________________________________________________________
gru_2 (GRU)                  (None, 34, 256)           393984    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 34, 256)           65792     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 34, 40)            10280     
Total para

In [22]:
BATCH_SIZE = 64
model.fit(train, 
          labels,
              batch_size=BATCH_SIZE,
              epochs=2)

Epoch 1/2
89056/89056 [==============================] - 641s 7ms/step - loss: 1.1889 - acc: 0.6480


Epoch 2/2
89056/89056 [==============================] - 648s 7ms/step - loss: 0.5639 - acc: 0.8196


In [23]:
BATCH_SIZE = 256
model.fit(train, labels,
              batch_size=BATCH_SIZE,
              epochs=2)

Epoch 1/2
89056/89056 [==============================] - 164s 2ms/step - loss: 0.4071 - acc: 0.8663
Epoch 2/2
88064/89056 [============================>.] - ETA: 1s - loss: 0.3687 - acc: 0.8780

89056/89056 [==============================] - 163s 2ms/step - loss: 0.3687 - acc: 0.8780


In [24]:
BATCH_SIZE = 1024
model.fit(train, labels,
              batch_size=BATCH_SIZE,
              epochs=2)

Epoch 1/2
89056/89056 [==============================] - 67s 755us/step - loss: 0.3218 - acc: 0.8931
Epoch 2/2
89056/89056 [==============================] - 67s 757us/step - loss: 0.3065 - acc: 0.8976


In [25]:
BATCH_SIZE = 4096
model.fit(train, labels,
              batch_size=BATCH_SIZE,
              epochs=3)

Epoch 1/3
89056/89056 [==============================] - 61s 689us/step - loss: 0.2918 - acc: 0.9024
Epoch 2/3
89056/89056 [==============================] - 60s 670us/step - loss: 0.2864 - acc: 0.9042
Epoch 3/3
89056/89056 [==============================] - 60s 671us/step - loss: 0.2833 - acc: 0.9050


In [0]:
drive.CreateFile({'id': '1MbRXFQFBHb_l5idofLcoxin-82On71gN'}).GetContentFile('test.csv')

In [0]:
df = pd.read_csv('test.csv')

In [28]:
df.head(10)

,Id,Word
0,1,PITCHED
1,2,DISSOLVERS
2,3,SCRAWNY
3,4,BONENFANT
4,5,EXCEEDS
5,6,BARTNICKI
6,7,BUTE
7,8,CAPITULATE
8,9,STEAM
9,10,INVESTCORP


In [0]:
test = np.zeros((len(df), MAX_INPUT_LEN, len(alphabet)))
i = 0
for word in df['Word']:
  j = 0
  for letter in word:
    test[i, j] = np.array([int(i == incoder_alph[letter]) for i in range(len(alphabet))])
    j += 1
  i += 1

In [30]:
np.argmax(test[1], axis = -1)

array([ 4,  9, 19, 19, 15, 12, 22,  5, 18, 19,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [31]:
res = model.predict(test, verbose = 1, batch_size=4096)

41597/41597 [==============================] - 8s 203us/step


In [0]:
ans = []
for word_emb in res:
  phonemes = []
  pred = np.argmax(word_emb, axis = 1)
  for i in range(len(pred)):
    if (pred[i] == 0 or (i != 0 and pred[i - 1] == pred[i])):
      break
    phonemes.append(list_phonems[pred[i]])
  ans.append('_'.join(phonemes))

In [39]:
ans[0]

'P_IH_CH_T'

In [0]:
def saver(path, res):
    with open(path, 'w') as out:
        print('Id,Transcription', file = out)
        for i in range(len(res)):
            print(str(i + 1) + ',' + res[i], file = out)

In [0]:
saver('last.csv', ans)

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
from googleapiclient.http import MediaFileUpload

def saver_inside(name):
  file_metadata = {
    'name': name + '.csv',
    'mimeType': 'text/csv'
  }
  media = MediaFileUpload(name + '.csv', 
                          mimetype='text/csv',
                          resumable=True)
  created = drive_service.files().create(body=file_metadata,
                                         media_body=media).execute()
  print('File ID: {}'.format(created.get('id')))

In [38]:
saver_inside('last')

File ID: 1hLs52X8mxdteW41W0sGSA1LBV_0mNv_x
